In [47]:
import tensorflow_datasets as tfds
import numpy as np
import cv2
from tensorflow.keras import layers
import tensorflow.keras as keras
import tensorflow

dataset_name = 'horses_or_humans'

(training_images, training_labels), (test_images, test_labels) =  tfds.as_numpy(tfds.load(dataset_name, split = ['train', 'test'], batch_size=-1, as_supervised=True))

In [48]:

input_img = layers.Input(shape=(300, 300, 1))

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='linear', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 1)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 300, 300, 128)     1280      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 150, 150, 128)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 150, 150, 32)      36896     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 75, 75, 8)         2312      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 38, 38, 8)         0   

In [49]:
train= [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)/255. for i in training_images]
test= [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)/255. for i in test_images]

In [50]:
train=np.expand_dims(np.array(train),axis=3)
test=np.expand_dims(np.array(test),axis=3)

In [51]:
print(train.shape)

(1027, 300, 300, 1)


In [56]:
es=tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3, restore_best_weights=True)
autoencoder.fit(train, train,epochs=10, batch_size= 32,shuffle=True,validation_data=(test, test), callbacks=[es])

Epoch 1/10
33/33 [==============================] - 624s 19s/step - loss: 0.0326 - val_loss: 0.0439
Epoch 2/10
33/33 [==============================] - 609s 18s/step - loss: 0.0109 - val_loss: 0.0158
Epoch 3/10
33/33 [==============================] - 591s 18s/step - loss: 0.0066 - val_loss: 0.0084
Epoch 4/10
33/33 [==============================] - 537s 16s/step - loss: 0.0048 - val_loss: 0.0082
Epoch 5/10
33/33 [==============================] - 500s 15s/step - loss: 0.0040 - val_loss: 0.0058
Epoch 6/10
33/33 [==============================] - 491s 15s/step - loss: 0.0036 - val_loss: 0.0059
Epoch 7/10
33/33 [==============================] - 495s 15s/step - loss: 0.0035 - val_loss: 0.0052
Epoch 8/10
33/33 [==============================] - 492s 15s/step - loss: 0.0033 - val_loss: 0.0048
Epoch 9/10
33/33 [==============================] - 571s 17s/step - loss: 0.0032 - val_loss: 0.0053
Epoch 10/10
33/33 [==============================] - 633s 19s/step - loss: 0.0033 - val_loss: 0.0049

In [57]:
autoencoder.save("autoencoder1.h5")

In [58]:
model=tensorflow.keras.models.load_model("autoencoder1.h5")

In [59]:
model=tensorflow.keras.models.Model(model.inputs, model.get_layer("max_pooling2d_8").output)

In [60]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 1)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 300, 300, 128)     1280      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 150, 150, 128)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 150, 150, 32)      36896     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 75, 75, 8)         2312      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 38, 38, 8)         0   

In [64]:
train_features=model.predict(train)
test_features=model.predict(test)

In [65]:
print(train_features.shape)

(1027, 38, 38, 8)


In [62]:
train_features=train_features
test_features=train_features

In [66]:
all_features=np.append(train_features,test_features,axis=0)
all_features= all_features.reshape(all_features.shape[0],-1)
all_features.shape

(1283, 11552)

In [67]:
labs=np.append(training_labels,test_labels,axis=0)
labs= labs.reshape(labs.shape[0],-1)
labs.shape

(1283, 1)

In [68]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(all_features)
labels=kmeans.labels_

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(labs, labels)

0.544037412314887